In [1]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

c:\users\bisite\anaconda3\envs\deeptrends\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
print(len(documents))
print(documents[:5])

1226258
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

c:\users\bisite\anaconda3\envs\deeptrends\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\BISITE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [5]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [6]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [7]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [8]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [9]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [10]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


In [11]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


In [12]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [13]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"death" + 0.026*"case" + 0.025*"charg" + 0.025*"court" + 0.021*"murder" + 0.020*"polic" + 0.015*"alleg" + 0.013*"trial" + 0.012*"arrest" + 0.012*"face"
Topic: 1 
Words: 0.022*"news" + 0.020*"market" + 0.017*"world" + 0.017*"women" + 0.015*"australian" + 0.015*"final" + 0.014*"island" + 0.012*"return" + 0.011*"street" + 0.010*"fall"
Topic: 2 
Words: 0.066*"coronavirus" + 0.037*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast" + 0.016*"restrict" + 0.014*"water" + 0.012*"gold" + 0.011*"plan" + 0.010*"park"
Topic: 3 
Words: 0.039*"sydney" + 0.026*"polic" + 0.021*"crash" + 0.020*"adelaid" + 0.019*"die" + 0.015*"miss" + 0.012*"break" + 0.011*"drug" + 0.011*"driver" + 0.010*"shoot"
Topic: 4 
Words: 0.038*"year" + 0.031*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail" + 0.015*"work" + 0.014*"high" + 0.014*"life" + 0.013*"offic" + 0.013*"interview"
Topic: 5 
Words: 0.029*"govern" + 0.020*"health" + 0.019*"school" + 0.017*"help" + 0.016*"chang" + 0.015*"fe

In [14]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.010*"guilti" + 0.010*"morrison" + 0.010*"weather" + 0.009*"monday" + 0.009*"border" + 0.009*"kill" + 0.008*"violenc" + 0.008*"christma" + 0.007*"bushfir" + 0.007*"juli"
Topic: 1 Word: 0.033*"trump" + 0.019*"donald" + 0.009*"wednesday" + 0.008*"hill" + 0.008*"care" + 0.008*"age" + 0.007*"lockdown" + 0.007*"peter" + 0.007*"alan" + 0.007*"social"
Topic: 2 Word: 0.011*"world" + 0.010*"scott" + 0.009*"australia" + 0.009*"thursday" + 0.008*"john" + 0.008*"leagu" + 0.007*"cricket" + 0.006*"daniel" + 0.006*"win" + 0.005*"celebr"
Topic: 3 Word: 0.018*"polic" + 0.016*"charg" + 0.015*"murder" + 0.011*"court" + 0.011*"alleg" + 0.011*"woman" + 0.010*"death" + 0.009*"jail" + 0.009*"arrest" + 0.008*"sentenc"
Topic: 4 Word: 0.012*"elect" + 0.011*"govern" + 0.009*"labor" + 0.008*"andrew" + 0.007*"liber" + 0.007*"say" + 0.007*"michael" + 0.007*"royal" + 0.007*"turnbul" + 0.006*"commiss"
Topic: 5 Word: 0.014*"countri" + 0.010*"hour" + 0.009*"restrict" + 0.009*"health" + 0.009*"coronaviru

In [15]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [16]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.41032394766807556	 
Topic: 0.029*"govern" + 0.020*"health" + 0.019*"school" + 0.017*"help" + 0.016*"chang" + 0.015*"feder" + 0.013*"indigen" + 0.012*"state" + 0.012*"communiti" + 0.012*"fund"

Score: 0.3498636782169342	 
Topic: 0.025*"tasmania" + 0.025*"call" + 0.020*"rise" + 0.019*"victorian" + 0.017*"morrison" + 0.017*"tasmanian" + 0.015*"million" + 0.015*"farm" + 0.011*"program" + 0.011*"claim"

Score: 0.15228082239627838	 
Topic: 0.070*"australia" + 0.044*"trump" + 0.024*"donald" + 0.017*"elect" + 0.016*"border" + 0.015*"busi" + 0.015*"peopl" + 0.014*"accus" + 0.013*"say" + 0.012*"scott"

Score: 0.01250617578625679	 
Topic: 0.066*"coronavirus" + 0.037*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast" + 0.016*"restrict" + 0.014*"water" + 0.012*"gold" + 0.011*"plan" + 0.010*"park"

Score: 0.012504493817687035	 
Topic: 0.038*"year" + 0.031*"melbourn" + 0.022*"open" + 0.021*"canberra" + 0.017*"jail" + 0.015*"work" + 0.014*"high" + 0.014*"life" + 0.013*"offic" + 0.013*"i

In [17]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4782187342643738	 
Topic: 0.012*"elect" + 0.011*"govern" + 0.009*"labor" + 0.008*"andrew" + 0.007*"liber" + 0.007*"say" + 0.007*"michael" + 0.007*"royal" + 0.007*"turnbul" + 0.006*"commiss"

Score: 0.4217502474784851	 
Topic: 0.014*"countri" + 0.010*"hour" + 0.009*"restrict" + 0.009*"health" + 0.009*"coronavirus" + 0.007*"climat" + 0.007*"chang" + 0.006*"mental" + 0.006*"fund" + 0.006*"farmer"

Score: 0.012504912912845612	 
Topic: 0.023*"coronavirus" + 0.018*"covid" + 0.016*"news" + 0.015*"market" + 0.012*"live" + 0.011*"rural" + 0.008*"australian" + 0.007*"australia" + 0.007*"share" + 0.006*"busi"

Score: 0.01250439416617155	 
Topic: 0.011*"north" + 0.009*"coast" + 0.008*"sport" + 0.007*"park" + 0.007*"david" + 0.006*"search" + 0.006*"korea" + 0.006*"island" + 0.006*"hobart" + 0.006*"aborigin"

Score: 0.012504267506301403	 
Topic: 0.033*"trump" + 0.019*"donald" + 0.009*"wednesday" + 0.008*"hill" + 0.008*"care" + 0.008*"age" + 0.007*"lockdown" + 0.007*"peter" + 0.007*"alan" +

In [18]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.3498365581035614	 Topic: 0.026*"death" + 0.026*"case" + 0.025*"charg" + 0.025*"court" + 0.021*"murder"
Score: 0.34103986620903015	 Topic: 0.022*"news" + 0.020*"market" + 0.017*"world" + 0.017*"women" + 0.015*"australian"
Score: 0.19239242374897003	 Topic: 0.042*"queensland" + 0.034*"victoria" + 0.022*"bushfir" + 0.022*"hous" + 0.014*"time"
Score: 0.016683056950569153	 Topic: 0.031*"china" + 0.026*"test" + 0.022*"coronavirus" + 0.020*"south" + 0.013*"north"
Score: 0.016676587983965874	 Topic: 0.025*"tasmania" + 0.025*"call" + 0.020*"rise" + 0.019*"victorian" + 0.017*"morrison"
Score: 0.016675863415002823	 Topic: 0.029*"govern" + 0.020*"health" + 0.019*"school" + 0.017*"help" + 0.016*"chang"
Score: 0.0166756771504879	 Topic: 0.066*"coronavirus" + 0.037*"covid" + 0.024*"live" + 0.021*"nation" + 0.020*"coast"
Score: 0.016673309728503227	 Topic: 0.039*"sydney" + 0.026*"polic" + 0.021*"crash" + 0.020*"adelaid" + 0.019*"die"
Score: 0.016673309728503227	 Topic: 0.038*"year" + 0.031*"m